In [ ]:
import requests
import pandas as pd

from io import StringIO
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta

def refreshFormData(soup, form_data):
    for elem in soup.find_all("input", {"type": "hidden"}):
        eid = elem.get("id")
        if "ctl00" in eid:
            eid = eid.replace("_", "$")
        form_data[eid] = elem.get("value")
    return form_data

form_data = {
    "__EVENTTARGET": "",
    "__EVENTARGUMENT": "",
    "__VIEWSTATE": "",
    "__VIEWSTATEGENERATOR": "",
    "__EVENTVALIDATION": "",
    "ctl00$ASCompany$hdnAutoSuggest": "0",
    "ctl00$ASCompany$txtAutoSuggest": "",
    "ctl00$txtNews": "",
    "ctl00$AutoSuggest1$hdnAutoSuggest": "0",
    "ctl00$AutoSuggest1$txtAutoSuggest": "",
    "ctl00$ContentPlaceHolder1$ASCompanyFilter$hdnAutoSuggest": "0",
    "ctl00$ContentPlaceHolder1$ASCompanyFilter$txtAutoSuggest": "",
    "ctl00$ContentPlaceHolder1$txtBuyerBrokerCodeFilter": "",
    "ctl00$ContentPlaceHolder1$txtSellerBrokerCodeFilter": "",
    "ctl00$ContentPlaceHolder1$txtFloorsheetDateFilter": "",
    "ctl00$ContentPlaceHolder1$PagerControl1$hdnPCID": "PC1",
    "ctl00$ContentPlaceHolder1$PagerControl1$hdnCurrentPage": "0",
    "ctl00$ContentPlaceHolder1$PagerControl1$btnPaging": "",
    "ctl00$ContentPlaceHolder1$PagerControl2$hdnPCID": "PC2",
    "ctl00$ContentPlaceHolder1$PagerControl2$hdnCurrentPage": "0"
}

In [ ]:
SESSION  = requests.session()
headers  = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"}
response = SESSION.get(url=r"https://merolagani.com/Floorsheet.aspx", headers=headers)
print(response.ok)

In [ ]:
DFS = []
# start_date = date.today() - timedelta(days=1)
start_date = datetime.strptime("09/10/2024", "%m/%d/%Y").date()
form_data = refreshFormData(BeautifulSoup(response.text, "html.parser"), form_data)

for i in range(100):
    current_date = start_date - timedelta(days=i)
    if current_date.weekday() != 5:
        if current_date.month != 9:
            break

        form_data["ctl00$ContentPlaceHolder1$txtFloorsheetDateFilter"] = str(current_date.strftime("%m/%d/%Y"))
        form_data["ctl00$ContentPlaceHolder1$PagerControl1$hdnCurrentPage"] = 1

        response = SESSION.post(r"https://merolagani.com/Floorsheet.aspx", data=form_data, headers=headers)
        print(current_date.strftime("%m/%d/%Y"), 1, response.status_code)
        
        soup  = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table")
        span  = soup.find("span", {"id":"ctl00_ContentPlaceHolder1_PagerControl1_litRecords"})

        if not table: continue
        DFS.append(pd.read_html(StringIO(str(table)))[0])

        page_count = int(span.text.split(" ")[-1].split("]")[0])

        for j in range(2, page_count+1):
            form_data["ctl00$ContentPlaceHolder1$txtFloorsheetDateFilter"] = current_date.strftime("%m/%d/%Y")
            form_data["ctl00$ContentPlaceHolder1$PagerControl1$hdnCurrentPage"] = j

            response = SESSION.post(r"https://merolagani.com/Floorsheet.aspx", data=form_data, headers=headers)
            print(current_date.strftime("%m/%d/%Y"), j, response.status_code)            

            soup = BeautifulSoup(response.text, "html.parser")
            DFS.append(pd.read_html(StringIO(str(soup.find("table"))))[0])

In [ ]:
pd.concat(DFS, ignore_index=True).to_parquet("0003.parquet")